<h1>CNN-H1 4-19-2016</h1>

<strong>Abstract</strong>
Implementing the CNN-H1 using NN2 described in the paper: http://arxiv.org/pdf/1509.00244v1.pdf. 

<strong>Improvements</strong>
<ul>
<li>Narrowed the width of faces to focus on the face, cut out background</li>
<li>Validation split of 15%</li>
<li>fc6 layer has a length of 128</li>
</ul>

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%matplotlib inline

import os
import fnmatch
import numpy as np

from skimage import io
from skimage.color import rgb2grey
from skimage.transform import resize

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.models import model_from_json

np.random.seed(123456)

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 98.0% of memory, CuDNN 3007)


In [26]:
data_path = '../data/lfw_cropped'

img_rows_load, img_cols_load = 160, 160
img_rows, img_cols = 160, 120
num_people = 600

<h2>Loading Files</h2>

In [4]:
def get_face_to_file_path_dict():
    face_to_file_paths_dict = {}
    
    for root, dirnames, filenames in os.walk(data_path):
        for dirname in dirnames:
            if dirname not in face_to_file_paths_dict:
                face_to_file_paths_dict[dirname] = []
            directory_path = os.path.join(data_path, dirname)
            for filename in os.listdir(directory_path):
                if filename.endswith(".jpg"):
                    face_to_file_paths_dict[dirname].append(os.path.join(directory_path, filename))
                            
    return face_to_file_paths_dict

In [5]:
def get_face_to_file_paths_descending_list(face_to_file_paths_dict):
    return sorted(face_to_file_paths_dict.items(), key=lambda x: len(x[1]), reverse=True)

In [6]:
face_to_file_paths_dict = get_face_to_file_path_dict()

In [7]:
face_to_file_paths_list = get_face_to_file_paths_descending_list(face_to_file_paths_dict)[:num_people]

<h2>Data Pre-Processing</h2>

In [20]:
def image_read(f):
    return resize(rgb2grey(io.imread(f)), (img_rows_load, img_cols_load))

def reflection(image):
    return np.array([list(reversed(row)) for row in image])

def partition(image, top_left, rows, cols):
    return np.array([row[top_left[1]:top_left[1] + cols] for row in image[top_left[0]:top_left[0] + rows]])

In [10]:
images_by_class = [[image_read(f) for f in x[1]] for x in face_to_file_paths_list]

In [11]:
for i in range(len(images_by_class)):
    images_by_class[i] += [reflection(im) for im in images_by_class[i]]

In [21]:
for i in range(len(images_by_class)):
    images_by_class[i] = [partition(im, (0, 20), img_rows, img_cols) for im in images_by_class[i]]

In [22]:
X_train = np.array([image for images in images_by_class for image in images])
y_train = np.array([images[0] for images in enumerate(images_by_class) for image in images[1]])

In [23]:
zipped = np.array(zip(X_train, y_train))
np.random.shuffle(zipped)

In [24]:
X_train = np.array([x[0] for x in zipped])
y_train = np.array([x[1] for x in zipped])

In [27]:
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
Y_train = np_utils.to_categorical(y_train, len(images_by_class))

<h2> Training and Validation</h2>

In [28]:
def NN2(input_shape, nb_classes, nb_fc6):
    model = Sequential()
    
    # Layer 1
    model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=input_shape))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 2
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 3
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 4
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 5
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3))
    model.add(ZeroPadding2D((1,1)))
    model.add(AveragePooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(nb_fc6))
    model.add(Dense(nb_classes, activation='softmax'))
   
    return model

In [29]:
input_shape = (1, img_rows, img_cols)
nb_fc6 = 128

In [30]:
model = NN2(input_shape, num_people, nb_fc6)
model.compile(loss='categorical_crossentropy', optimizer='sgd')
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10, 
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

Train on 11352 samples, validate on 2004 samples
Epoch 1/10
11352/11352 [==============================] - 354s - loss: 6.0091 - acc: 0.0750 - val_loss: 5.8350 - val_acc: 0.0843
Epoch 2/10
11352/11352 [==============================] - 354s - loss: 5.8082 - acc: 0.0782 - val_loss: 5.7898 - val_acc: 0.0843
Epoch 3/10
11352/11352 [==============================] - 354s - loss: 5.7664 - acc: 0.0775 - val_loss: 5.7692 - val_acc: 0.0843
Epoch 4/10
11352/11352 [==============================] - 354s - loss: 5.7508 - acc: 0.0785 - val_loss: 5.7761 - val_acc: 0.0843
Epoch 5/10
11352/11352 [==============================] - 354s - loss: 5.7407 - acc: 0.0783 - val_loss: 5.8062 - val_acc: 0.0843
Epoch 6/10
11352/11352 [==============================] - 354s - loss: 5.7246 - acc: 0.0783 - val_loss: 5.7389 - val_acc: 0.0843
Epoch 7/10
11352/11352 [==============================] - 353s - loss: 5.6958 - acc: 0.0782 - val_loss: 5.6843 - val_acc: 0.0843
Epoch 8/10
11352/11352 [========================

In [31]:
json_string = model.to_json()
open('models/CNN-H1-fc6-128-10e.json', 'w').write(json_string)
model.save_weights('models/CNN-H1-fc6-128-10e.h5', overwrite=True)

In [32]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001))
model.fit(X_train, Y_train, batch_size=32, nb_epoch=100,
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

Train on 11352 samples, validate on 2004 samples
Epoch 1/100
11352/11352 [==============================] - 354s - loss: 4.9456 - acc: 0.1759 - val_loss: 5.0023 - val_acc: 0.1881
Epoch 2/100
11352/11352 [==============================] - 354s - loss: 4.7977 - acc: 0.1958 - val_loss: 4.9032 - val_acc: 0.2006
Epoch 3/100
11352/11352 [==============================] - 354s - loss: 4.7062 - acc: 0.2052 - val_loss: 4.8093 - val_acc: 0.2081
Epoch 4/100
11352/11352 [==============================] - 354s - loss: 4.5967 - acc: 0.2179 - val_loss: 4.7832 - val_acc: 0.2196
Epoch 5/100
11352/11352 [==============================] - 354s - loss: 4.5032 - acc: 0.2253 - val_loss: 4.6240 - val_acc: 0.2360
Epoch 6/100
11352/11352 [==============================] - 354s - loss: 4.3814 - acc: 0.2445 - val_loss: 4.5420 - val_acc: 0.2445
Epoch 7/100
11352/11352 [==============================] - 354s - loss: 4.2613 - acc: 0.2557 - val_loss: 4.5127 - val_acc: 0.2485
Epoch 8/100
11352/11352 [================

In [33]:
json_string = model.to_json()
open('models/CNN-H1-fc6-128-110.json', 'w').write(json_string)
model.save_weights('models/CNN-H1-fc6-128-110e.h5', overwrite=True)